In [43]:
import parmed as pmd
from pdbx_parser import *
import numpy as np
import copy

In [2]:
def calculate_distance(x1, y1, z1, x2, y2, z2):
    # 计算距离的公式为欧氏距离
    distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
    return distance

In [3]:
class Mol():
    def __init__(self, data):
        self.data = data
    
    def __getitem__(self, index):
        return self.data[index]
    
    def __setiterm__(self, index, value):
        self.data[index] = value
        
    def __len__(self):
        return len(self.data)

In [44]:
class Gener_Atom():
    def __init__(self, pdbx_atom, parmed_atom, ref_dist=None):
        self.pdbx_atom = pdbx_atom
        self.parmed_atom = parmed_atom
        self.ref_dist = ref_dist
        
        self.pdbx_atomid = self.pdbx_atom.atomid
        self.parmed_atomidx = self.parmed_atom.idx
        self.pdbx_parmed_idx_gap = self.pdbx_atomid - self.parmed_atomidx

        self.bonded_H_parmed_atm = [ i for i in self.parmed_atom.bond_partners if i.element_name == 'H']
        self.bonded_H_parmed_idx = [ i.idx for i in self.bonded_H_parmed_atm ]
        self.bonded_H_pdbx_id = [i.idx+self.pdbx_parmed_idx_gap for i in self.bonded_H_parmed_atm]

        self.heavy_atom_and_H_parmed_idx = copy.deepcopy(self.bonded_H_parmed_idx)
        self.heavy_atom_and_H_parmed_idx.append(self.parmed_atomidx)
        self.heavy_atom_and_H_pdbx_id = copy.deepcopy(self.bonded_H_pdbx_id)
        self.heavy_atom_and_H_pdbx_id.append(self.pdbx_atomid)

        

In [45]:
class Group_generator():
    def __init__(self, top_file, xyz_file, pdbx_file):
        self.top_file = top_file
        self.xyz_file = xyz_file
        self.pdbx = PdbxParser(pdbx_file)
        self.parm = pmd.load_file(top_file, xyz=xyz_file)
        self.pdbx_start_idx = self.pdbx.atoms_list[0].atomid
        self.group_dict = {}
        self.gen_Mol_obj()
        

    def gen_Mol_obj(self,):
        atom_data = []
        for pdbx_atom in self.pdbx.atoms_list:
            parmed_atom = self.parm.atoms[pdbx_atom.atomid-self.pdbx_start_idx]
            one_gener_atom = Gener_Atom(pdbx_atom, parmed_atom)
            atom_data.append(one_gener_atom)
        self.mol = Mol(atom_data)
    
    def gen_grp_by_per_atom(self, last_grp):
        '''
        last_grp is a list like [atom1_id, atom2_id, ...], atom_id is the atomid record in the pdbx file.
        '''
        # convert pdbx_file atomid to ParmEd atom.idx(start from 0)
        _last_grp = [i-self.pdbx_start_idx for i in last_grp]
#         print(_last_grp)
        last_grp_incl_H_gener_atoms_idx = []
        for idx in _last_grp:
            last_grp_incl_H_gener_atoms_idx+=self.mol[idx].heavy_atom_and_H_parmed_idx
        
        last_grp_coords = []
        for idx in last_grp_incl_H_gener_atoms_idx:
            coord_array = np.array(self.mol[idx].pdbx_atom.coord)
            last_grp_coords.append(coord_array)
        last_grp_mass_coord = np.vstack(last_grp_coords).sum(axis=0)/len(last_grp_coords)
        x1, y1, z1 = last_grp_mass_coord
        unlast_grp_heavy_atom = []
        for gener_atom in self.mol.data:
            if gener_atom.parmed_atom.idx not in last_grp_incl_H_gener_atoms_idx and gener_atom.parmed_atom.element_name != 'H':
                x2, y2, z2 = gener_atom.pdbx_atom.coord
                gener_atom.ref_dist = calculate_distance(x1, y1, z1, x2, y2, z2)
                unlast_grp_heavy_atom.append(gener_atom)
        sorted_unlast_grp_heavy_atom = sorted(unlast_grp_heavy_atom, key=lambda _gener_atom: _gener_atom.ref_dist)
        
        _idx = 1
        for heavy_atom in sorted_unlast_grp_heavy_atom:
            self.group_dict[f'group{_idx}'] = heavy_atom.heavy_atom_and_H_pdbx_id
            _idx+=1
        self.group_dict[f'group{len(sorted_unlast_grp_heavy_atom)+1}'] = [i+self.pdbx_start_idx for i in last_grp_incl_H_gener_atoms_idx]
        return self.group_dict
    

In [46]:
top_file='lig.prmtop'
xyz_file='lig.prmcrd'
pdbx_file = 'mol.pdbx'
a = Group_generator(top_file, xyz_file, pdbx_file)

In [47]:
a.mol[29].bonded_H_pdbx_id

[5112]

In [48]:
a.gen_grp_by_per_atom([5111,])

{'group1': [5109],
 'group2': [5113],
 'group3': [5108],
 'group4': [5107],
 'group5': [5127],
 'group6': [5116],
 'group7': [5110],
 'group8': [5117],
 'group9': [5115, 5114],
 'group10': [5093],
 'group11': [5098],
 'group12': [5128],
 'group13': [5129],
 'group14': [5118],
 'group15': [5104, 5105, 5100],
 'group16': [5092, 5091],
 'group17': [5106, 5099],
 'group18': [5095, 5094],
 'group19': [5097, 5096],
 'group20': [5124, 5125, 5126, 5123],
 'group21': [5120, 5121, 5122, 5119],
 'group22': [5102, 5103, 5101],
 'group23': [5131, 5132, 5133, 5130],
 'group24': [5090],
 'group25': [5084, 5083],
 'group26': [5089, 5088],
 'group27': [5082],
 'group28': [5087],
 'group29': [5086, 5085],
 'group30': [5112, 5111]}

In [50]:

from optparse import OptionParser

class optParser():
    def __init__(self, fakeArgs):
        parser = OptionParser()
        parser.add_option('-p', '--top', dest='top', help='The name of the topology file.', default='lig.prmtop')
        parser.add_option('-c', '--crd', dest='crd', help='The name of the coordinate file.', default='lig.prmcrd')
        parser.add_option('-x', '--pdbx', dest='pdbx', help='The name of the pdbx file.', default='ini_lig.pdbx')
        parser.add_option('-l', '--last_grp_atom', dest='last_grp_atom', help='The list of the last_grp_atom.', )

        if fakeArgs:
            self.option, self.args = parser.parse_args(fakeArgs)
        else:
            self.option, self.args = parser.parse_args()

In [51]:
opts = optParser('-p top -c crd -x pdbx -l [0,1,2]'.strip().split())

In [55]:
eval('True')

True